In [1]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=a08aecd1b6a84054a5db3bf9ac382649e0f8f2a267fe07635c7767395ac38224
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [44]:
# import required libraries
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
import seaborn as sns

In [537]:
stop_loss_multiplier = 1.0
risk_reward_ratio = 2.0
set_adx_value = 25

In [538]:
# !nvidia-smi

In [560]:
df = pd.read_csv('ETHUSD_M15.csv')
df.head()

,Date,Open,High,Low,Close,Vol
0,06/10/2021 12:45,3436.2,3455.9,3426.7,3453.6,1
1,06/10/2021 13:00,3453.6,3572.5,3441.1,3539.5,1
2,06/10/2021 13:15,3539.1,3567.9,3511.1,3551.1,1
3,06/10/2021 13:30,3551.1,3578.6,3528.0,3578.6,1
4,06/10/2021 13:45,3578.7,3619.7,3557.2,3574.7,2


In [561]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    100000 non-null  object 
 1   Open    100000 non-null  float64
 2   High    100000 non-null  float64
 3   Low     100000 non-null  float64
 4   Close   100000 non-null  float64
 5   Vol     100000 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB


In [562]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    100000 non-null  datetime64[ns]
 1   Open    100000 non-null  float64       
 2   High    100000 non-null  float64       
 3   Low     100000 non-null  float64       
 4   Close   100000 non-null  float64       
 5   Vol     100000 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 4.6 MB


In [563]:
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Vol
Date,,,,,
2021-10-06 12:45:00,3436.2,3455.9,3426.7,3453.6,1
2021-10-06 13:00:00,3453.6,3572.5,3441.1,3539.5,1
2021-10-06 13:15:00,3539.1,3567.9,3511.1,3551.1,1
2021-10-06 13:30:00,3551.1,3578.6,3528.0,3578.6,1
2021-10-06 13:45:00,3578.7,3619.7,3557.2,3574.7,2


In [564]:
# now calculate the lower and upper bb band with a look back of 20, and a stddev of 2.0.
# calculate the body size of each candle
# calculate the maximum wick size of each candle

In [565]:
# Calculate the moving average (middle band)
lookback_period = 20
df['MiddleBand'] = df['Close'].rolling(window=lookback_period).mean()
# Calculate the standard deviation
df['StandardDeviation'] = df['Close'].rolling(window=lookback_period).std()
# Calculate the upper and lower Bollinger Bands
stddev_multiplier = 2.0
df['UpperBand'] = df['MiddleBand'] + (stddev_multiplier * df['StandardDeviation'])
df['LowerBand'] = df['MiddleBand'] - (stddev_multiplier * df['StandardDeviation'])

In [566]:
df['BodySize'] = abs(df['Close'] - df['Open'])

In [567]:
# Calculate the upper wick size
df['UpperWickSize'] = df['High'] - df[['Open', 'Close']].max(axis=1)

# Calculate the lower wick size
df['LowerWickSize'] = df[['Open', 'Close']].min(axis=1) - df['Low']

# Calculate the maximum wick size
df['MaxWickSize'] = df[['UpperWickSize', 'LowerWickSize']].max(axis=1)

In [568]:
# check whether the body_size is greater than max wick size
df['momentum'] = (df['BodySize'] >= df['MaxWickSize']).astype(int)
# df.head(30)

In [569]:
# add adx values and adx condition
df['adx_value'] = ta.trend.ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx()
# df.head(30)
# df['adx'] = np.where((df['adx_value'] < set_adx_value), 1, 0)
df['adx'] = np.where((df['adx_value'] < set_adx_value) & (df['momentum'] == 1), 1, 0)
# df.head(30)


In [570]:
# for each candle, check whether the whole body is greater than the upper_bollinger_band and if true return 1, whether the whole body is less than the lower bollinger
# band and return 2 and else return 0

df['BodyVsBollinger'] = 0
# Check if both Open and Close are above the Upper Band
df.loc[(df['Open'] >= df['UpperBand']) & (df['Close'] >= df['UpperBand']), 'BodyVsBollinger'] = 1
# Check if both Open and Close are below the Lower Band
df.loc[(df['Open'] <= df['LowerBand']) & (df['Close'] <= df['LowerBand']), 'BodyVsBollinger'] = 2

# Display the updated DataFrame
# df.head(30)

In [571]:
(df['BodyVsBollinger'] == 1).sum(), (df['BodyVsBollinger'] == 2).sum()

(1314, 1293)

In [572]:
# first we compute the profitability of the system based on manipulation without considering the wick of the body
df['pureSignal'] = np.where((df['adx']==1) & (df['BodyVsBollinger']==1), 1,
                            np.where((df['adx']==1) & (df['BodyVsBollinger']==2), 2, 0))
# df.head(30)

In [573]:
(df['pureSignal'] == 1).sum(), (df['pureSignal'] == 2).sum()

(144, 134)

## ADD Trade Parameters
* Calculate the upper stop loss and the lower stop loss. Also, calculate the upper take profit and the lower take profit.
* To calculate the stop loss value, we use the atr based method. We will define the atr multiplier to get the stop loss of both upper and below!


In [574]:
# Calculate ATR
atr_period = 14  # ATR period
df['ATR'] = ta.volatility.AverageTrueRange(
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    window=atr_period
).average_true_range()
# Define a multiplier for the stop loss
# drop all null values
df.dropna(inplace=True)
df['LowerStopLoss'] = df['Low'] - (df['ATR'] * stop_loss_multiplier)
df['UpperStopLoss'] = df['High'] + (df['ATR'] * stop_loss_multiplier)

<i> Now that we have the upper and lower stop loss, our next task is to dynamically add the upper and lower stop loss ==> an even more challenging task.
* For lower take profit, we  take upperstoploss - df.close. We then multiply this distance by risk-reward-ratio. We then take the close value of the candle and subtract this distance.

* For upper take profit, we take the close - lower stop loss and multiply this value by risk-reward_multiplier. we then take the close and add this value.

In [575]:
df['LowerTakeProfit'] = df['Close'] - ((df['UpperStopLoss'] - df['Close']) * risk_reward_ratio)
df['UpperTakeProfit'] = df['Close'] + ((df['Close'] - df['LowerStopLoss']) * risk_reward_ratio)
# df.head(30)

In [576]:
# df.columns

In [577]:
short_trades, long_trades = (df['pureSignal']==1).sum(), (df['pureSignal']==2).sum()
short_trades, long_trades # when we have not added the momentum factor into our trades instead we have only added adx

(144, 134)

* <i> now that we have everything that we need to take our signals, it is time to evaluate whether when 1, the trades hits take profit or the trade hits stop loss. Same when the signal is 2. Does the trade hits the upper take profit or does it hit the lower stop loss </i>


In [ ]:
# LowerTakeProfit,UpperStopLoss,UpperTakeProfit,LowerStopLoss

In [578]:
def calculate_final_result(df):
    df['final_result'] = 0  # Initialize with 0

    for i in range(len(df)):
        current_index = df.index[i]
        if df.loc[current_index, 'pureSignal'] == 1:
            for j in range(i+1, len(df)):
                future_index = df.index[j]
                if df.loc[future_index, 'Low'] <= df.loc[current_index, 'LowerTakeProfit']:
                    df.loc[current_index, 'final_result'] = 1
                    break
                elif df.loc[future_index, 'High'] >= df.loc[current_index, 'UpperStopLoss']:
                    df.loc[current_index, 'final_result'] = 0
                    break
        elif df.loc[current_index, 'pureSignal'] == 2:
            for j in range(i+1, len(df)):
                future_index = df.index[j]
                if df.loc[future_index, 'High'] >= df.loc[current_index, 'UpperTakeProfit']:
                    df.loc[current_index, 'final_result'] = 2
                    break
                elif df.loc[future_index, 'Low'] <= df.loc[current_index, 'LowerStopLoss']:
                    df.loc[current_index, 'final_result'] = 0
                    break

    return df

# Assuming your DataFrame is called 'df'
df = calculate_final_result(df)

In [579]:
c_shorts, c_longs = (df['final_result']==1).sum(), (df['final_result']==2).sum()
print(short_trades, long_trades)
print(c_shorts, c_longs)

144 134
40 41


In [580]:
c_shorts/short_trades *100, c_longs/long_trades*100 # values in percentages

(27.77777777777778, 30.597014925373134)